
# Практика 3


## Выполнил студент группы ББМО-01-23 Егоров Юрий Александрович

### Пункт 1 


In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

# Загрузка тестовых данных MNIST
(_, _), (test_images, test_labels) = mnist.load_data()

# Нормализация данных
test_images = test_images / 255.0

# Загрузка обученной модели
model = tf.keras.models.load_model('mnist_model.h5')

# Компиляция модели с правильной функцией потерь
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Проверка точности модели на обычных данных
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy on clean images: {test_acc}')


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 525us/step - accuracy: 0.9733 - loss: 0.0970
Test accuracy on clean images: 0.9768000245094299


## 2. Теоретическая часть: Атака Carlini-Wagner
- Атака Carlini-Wagner (CW) оптимизирует изображение, минимально изменяя его, чтобы модель дала неверный результат. 
- Цель атаки — сделать модификации настолько незаметными, что изображение всё ещё выглядит как исходное, но классифицируется неправильно.
- Особенность CW атаки заключается в минимизации изменений с использованием разных норм (L2, L0, L∞), что делает её более точной по сравнению с другими атаками.

### Пункт 3 


In [4]:
!pip install foolbox==3.0.0



DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/jupyterlab_widgets-3.0.9-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/nbformat-5.9.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/jedi-0.19.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/ipykernel-6.25.2-py3.11.egg is deprecated. pip 24.3 will e

In [5]:
pip install --upgrade foolbox tensorflow


DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/jupyterlab_widgets-3.0.9-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/nbformat-5.9.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/jedi-0.19.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/ipykernel-6.25.2-py3.11.egg is deprecated. pip 24.3 will e

In [9]:
pip install --upgrade foolbox tensorflow eagerpy


DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/jupyterlab_widgets-3.0.9-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/nbformat-5.9.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/jedi-0.19.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/ipykernel-6.25.2-py3.11.egg is deprecated. pip 24.3 will e

In [10]:
pip install tensorflow==2.10


DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/jupyterlab_widgets-3.0.9-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/nbformat-5.9.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/jedi-0.19.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/ipykernel-6.25.2-py3.11.egg is deprecated. pip 24.3 will e

In [11]:
import foolbox as fb
import numpy as np
import matplotlib.pyplot as plt

# Инициализация Foolbox модели
fmodel = fb.TensorFlowModel(model, bounds=(0, 1))

# Создание атаки FGSM
attack = fb.attacks.FGSM()

# Выбор изображения для атаки (например, первое изображение из тестового набора)
image = test_images[0].astype(np.float32) / 255.0  # Нормализация и преобразование к float32
label = np.array([test_labels[0]], dtype=np.int32)  # Оберните в список для добавления новой размерности

# Выполнение атаки
adversarial_image = attack(fmodel, image.reshape(1, 28, 28, 1), label, epsilons=[0.1])

# Вывод результатов
plt.figure()
plt.subplot(1, 2, 1)
plt.title("Оригинальное изображение")
plt.imshow(image.squeeze(), cmap="gray")
plt.subplot(1, 2, 2)
plt.title("Противоречивое изображение (FGSM Attack)")
plt.imshow(adversarial_image[0].reshape(28, 28), cmap="gray")
plt.show()


NotImplementedError: 